In [1]:
import argparse
import os
import os.path as osp
import shutil
import time
from itertools import product
from json import dumps
import random
import numpy as np
import seaborn as sns
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch.optim import Adam
from torch_geometric.loader import DataListLoader, DataLoader
from torch_geometric.nn import DataParallel
from torch_geometric.seed import seed_everything
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import DataLoader, Data
from torch import nn
from tqdm import tqdm

from pytorch_metric_learning.losses import NTXentLoss, VICRegLoss
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from torch_geometric.datasets import GNNBenchmarkDataset
from datasets.SRDataset import SRDataset

import train_utils
from data_utils import extract_edge_attributes
from torch_geometric.datasets import TUDataset
from layers.input_encoder import LinearEncoder, LinearEdgeEncoder
from layers.layer_utils import make_gnn_layer
from models.GraphClassification import GraphClassification
from models.model_utils import make_GNN

import GCL.losses as L
import GCL.augmentors as A
from GCL.eval import get_split, SVMEvaluator, LREvaluator
from GCL.models import DualBranchContrast

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Feature Augmentation
parser = argparse.ArgumentParser(f'arguments for training and testing')
parser.add_argument('--save_dir', type=str, default='./save', help='Base directory for saving information.')
parser.add_argument('--seed', type=int, default=2, help='Random seed for reproducibility.')
parser.add_argument('--dataset_name', type=str, default="CSL", help='Name of dataset')
parser.add_argument('--drop_prob', type=float, default=0.6,
                    help='Probability of zeroing an activation in dropout layers.') 
parser.add_argument('--batch_size', type=int, default=32, help='Batch size per GPU. Scales automatically when \
                        multiple GPUs are available.')
parser.add_argument("--parallel", action="store_true",
                    help="If true, use DataParallel for multi-gpu training")
parser.add_argument('--num_workers', type=int, default=0, help='Number of worker.')
parser.add_argument('--load_path', type=str, default=None, help='Path to load as a model checkpoint.')
parser.add_argument('--lr', type=float, default=0.001, help='Learning rate.')
parser.add_argument('--l2_wd', type=float, default=5e-6, help='L2 weight decay.')
parser.add_argument('--num_epochs', type=int, default=500, help='Number of epochs.')
parser.add_argument("--hidden_size", type=int, default=200, help="Hidden size of the model")
parser.add_argument("--model_name", type=str, default="KHopGNNConv",
                    choices=("KHopGNNConv"), help="Base GNN model")
parser.add_argument("--K", type=int, default=3, help="Number of hop to consider")
parser.add_argument("--num_layer", type=int, default=3, help="Number of layer for feature encoder")
parser.add_argument("--JK", type=str, default="sum", choices=("sum", "max", "mean", "attention", "last", "concat"),
                    help="Jumping knowledge method")
parser.add_argument("--residual", default=True, action="store_true", help="If true, use residual connection between each layer")
parser.add_argument("--virtual_node", action="store_true", default=False, 
                    help="If true, add virtual node information in each layer")
parser.add_argument('--split', type=int, default=10, help='Number of fold in cross validation')
parser.add_argument("--train_eps", action="store_true", help="If true, the epsilon in GIN model is trainable")
parser.add_argument("--combine", type=str, default="geometric", choices=("attention", "geometric"),
                    help="Combine method in k-hop aggregation")
parser.add_argument("--pooling_method", type=str, default="sum", choices=("mean", "sum", "attention"),
                    help="Pooling method in graph classification")
parser.add_argument('--norm_type', type=str, default="Batch",
                    choices=("Batch", "Layer", "Instance", "GraphSize", "Pair"),
                    help="Normalization method in model")
parser.add_argument('--aggr', type=str, default="add",
                    help='Aggregation method in GNN layer, only works in GraphSAGE')
parser.add_argument("--patience", type=int, default=20, help="Patient epochs to wait before early stopping.")
parser.add_argument('--factor', type=float, default=0.5, help='Factor for reducing learning rate scheduler')
parser.add_argument('--reprocess', action="store_true", help='If true, reprocess the dataset')
parser.add_argument('--search', action="store_true", help='If true, search hyper-parameters')
parser.add_argument("--pos_enc_dim", type=int, default=6, help="Initial positional dim.")
parser.add_argument("--pos_attr", type=bool, default=False, help="Positional attributes.")
parser.add_argument("--feature_augmentation", type=bool, default=True, help="If true, feature augmentation.")

# Structure Augmentation
# parser = argparse.ArgumentParser(f'arguments for training and testing')
# parser.add_argument('--save_dir', type=str, default='./save', help='Base directory for saving information.')
# parser.add_argument('--seed', type=int, default=200, help='Random seed for reproducibility.')
# parser.add_argument('--dataset_name', type=str, default="CSL", help='Name of dataset')
# parser.add_argument('--drop_prob', type=float, default=0.6129,
#                     help='Probability of zeroing an activation in dropout layers.') 
# parser.add_argument('--batch_size', type=int, default=16, help='Batch size per GPU. Scales automatically when \
#                         multiple GPUs are available.')
# parser.add_argument("--parallel", action="store_true",
#                     help="If true, use DataParallel for multi-gpu training")
# parser.add_argument('--num_workers', type=int, default=0, help='Number of worker.')
# parser.add_argument('--load_path', type=str, default=None, help='Path to load as a model checkpoint.')
# parser.add_argument('--lr', type=float, default=0.0001, help='Learning rate.')
# parser.add_argument('--l2_wd', type=float, default=5e-4, help='L2 weight decay.')
# parser.add_argument('--num_epochs', type=int, default=500, help='Number of epochs.')
# parser.add_argument("--hidden_size", type=int, default=200, help="Hidden size of the model")
# parser.add_argument("--model_name", type=str, default="KHopGNNConv",
#                     choices=("KHopGNNConv"), help="Base GNN model")
# parser.add_argument("--K", type=int, default=2, help="Number of hop to consider")
# parser.add_argument("--num_layer", type=int, default=2, help="Number of layer for feature encoder")
# parser.add_argument("--JK", type=str, default="sum", choices=("sum", "max", "mean", "attention", "last", "concat"),
#                     help="Jumping knowledge method")
# parser.add_argument("--residual", default=True, action="store_true", help="If true, use residual connection between each layer")
# parser.add_argument("--virtual_node", action="store_true", default=False, 
#                     help="If true, add virtual node information in each layer")
# parser.add_argument('--split', type=int, default=10, help='Number of fold in cross validation')
# parser.add_argument("--train_eps", action="store_true", help="If true, the epsilon in GIN model is trainable")
# parser.add_argument("--combine", type=str, default="geometric", choices=("attention", "geometric"),
#                     help="Combine method in k-hop aggregation")
# parser.add_argument("--pooling_method", type=str, default="sum", choices=("mean", "sum", "attention"),
#                     help="Pooling method in graph classification")
# parser.add_argument('--norm_type', type=str, default="Batch",
#                     choices=("Batch", "Layer", "Instance", "GraphSize", "Pair"),
#                     help="Normalization method in model")
# parser.add_argument('--aggr', type=str, default="add",
#                     help='Aggregation method in GNN layer, only works in GraphSAGE')
# parser.add_argument("--patience", type=int, default=20, help="Patient epochs to wait before early stopping.")
# parser.add_argument('--factor', type=float, default=0.5, help='Factor for reducing learning rate scheduler')
# parser.add_argument('--reprocess', action="store_true", help='If true, reprocess the dataset')
# parser.add_argument('--search', action="store_true", help='If true, search hyper-parameters')
# parser.add_argument("--pos_enc_dim", type=int, default=6, help="Initial positional dim.")
# parser.add_argument("--pos_attr", type=bool, default=False, help="Positional attributes.")
# parser.add_argument("--feature_augmentation", type=bool, default=False, help="If true, feature augmentation.")

_StoreAction(option_strings=['--feature_augmentation'], dest='feature_augmentation', nargs=None, const=None, default=True, type=<class 'bool'>, choices=None, required=False, help='If true, feature augmentation.', metavar=None)

In [3]:
args = parser.parse_args("")

In [4]:
args.name = args.model_name + "_" + str(args.K) + "_" + str(args.search)

In [5]:
# Set up logging and devices
args.save_dir = train_utils.get_save_dir(args.save_dir, args.name, type=args.dataset_name)
log = train_utils.get_logger(args.save_dir, args.name)
device, args.gpu_ids = train_utils.get_available_devices()

In [6]:
if len(args.gpu_ids) > 1 and args.parallel:
    log.info(f'Using multi-gpu training')
    args.parallel = True
    loader = DataListLoader
    args.batch_size *= max(1, len(args.gpu_ids))
else:
    log.info(f'Using single-gpu training')
    args.parallel = False
    loader = DataLoader

[10.11.24 16:08:57] Using single-gpu training


In [7]:
# Set random seed
seed = args.seed
log.info(f'Using random seed {seed}...')
seed_everything(seed)

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

[10.11.24 16:08:57] Using random seed 2...


In [8]:
def edge_feature_transform(g):
    return extract_edge_attributes(g, args.pos_enc_dim)

In [9]:
tag = str(int(time.time()))

In [10]:
class LogReg(nn.Module):
    def __init__(self, hid_dim, out_dim):
        super(LogReg, self).__init__()

        self.fc = nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        ret = self.fc(x)
        return ret

In [11]:
def csl_train(loader, model, optimizer, device, parallel=False):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        if parallel:
            num_graphs = len(data)
            y = torch.cat([d.y for d in data]).to(device)
        else:
            num_graphs = data.num_graphs
            data = data.to(device)
            y = data.y
        out = model(data.graph_embeds).squeeze()
        loss = F.cross_entropy(out, y)
        loss.backward()
        total_loss += loss.item() * num_graphs
        optimizer.step()
    return total_loss / len(loader.dataset)

In [12]:
@torch.no_grad()
def csl_val(loader, model, device, parallel=False):
    model.eval()
    y_preds, y_trues = [], []
    for data in loader:
        if parallel:
            y = torch.cat([d.y for d in data]).to(device)
        else:
            data = data.to(device)
            y = data.y
        y_preds.append(torch.argmax(model(data.graph_embeds), dim=-1))
        y_trues.append(y)
    y_preds = torch.cat(y_preds, -1)
    y_trues = torch.cat(y_trues, -1)
    return (y_preds == y_trues).float().mean()

In [13]:
def get_model(args):
    layer = make_gnn_layer(args)
    init_emb = LinearEncoder(args.input_size, args.hidden_size, pos_attr=args.pos_attr)
    init_edge_attr_emb = LinearEdgeEncoder(args.edge_attr_size, args.hidden_size, edge_attr=True)
    init_edge_attr_v2_emb = LinearEdgeEncoder(args.edge_attr_v2_size, args.hidden_size, edge_attr=False)
    
    GNNModel = make_GNN(args)
    
    gnn = GNNModel(
        num_layer=args.num_layer,
        gnn_layer=layer,
        JK=args.JK,
        norm_type=args.norm_type,
        init_emb=init_emb,
        init_edge_attr_emb=init_edge_attr_emb,
        init_edge_attr_v2_emb=init_edge_attr_v2_emb,
        residual=args.residual,
        virtual_node=args.virtual_node,
        drop_prob=args.drop_prob)

    model = GraphClassification(embedding_model=gnn,
                                pooling_method=args.pooling_method,
                                output_size=args.output_size)
    
    model.reset_parameters()

    if args.parallel:
        model = DataParallel(model, args.gpu_ids)
    return model

In [14]:
class Projection(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Projection, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, output_dim),
            nn.ReLU()
        )
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x) + self.linear(x)

In [15]:
def vicreg_loss(embeddings1, embeddings2, lambda_var=25, lambda_cov=25, mu=1):
    """
    Calculate the VICReg loss between two sets of embeddings with the correct handling of covariance differences.
    
    Args:
    embeddings1, embeddings2 (torch.Tensor): Embeddings from two views, shape (batch_size, feature_dim).
    lambda_var (float): Coefficient for the variance loss.
    lambda_cov (float): Coefficient for the covariance loss.
    mu (float): Coefficient for the invariance loss.
    
    Returns:
    torch.Tensor: The total VICReg loss.
    """
    # Invariance Loss
    invariance_loss = F.mse_loss(embeddings1, embeddings2)

    # Variance Loss
    def variance_loss(embeddings1, embeddings2):
        mean_embeddings1 = embeddings1.mean(dim=0)
        mean_embeddings2 = embeddings2.mean(dim=0)
        
        std_dev1 = torch.sqrt((embeddings1 - mean_embeddings1).var(dim=0) + 1e-4)
        std_dev2 = torch.sqrt((embeddings2 - mean_embeddings2).var(dim=0) + 1e-4)
        
        return torch.mean(torch.abs(F.relu(1 - std_dev1) - F.relu(1 - std_dev2)))

    variance_loss_value = variance_loss(embeddings1, embeddings2)

    # Covariance Loss
    def covariance_loss(embeddings1, embeddings2):
        batch_size, feature_dim = embeddings1.size()
        
        embeddings_centered1 = embeddings1 - embeddings1.mean(dim=0)
        embeddings_centered2 = embeddings2 - embeddings2.mean(dim=0)
        
        covariance_matrix1 = torch.matmul(embeddings_centered1.T, embeddings_centered1) / (batch_size - 1)
        covariance_matrix2 = torch.matmul(embeddings_centered2.T, embeddings_centered2) / (batch_size - 1)
        
        covariance_matrix1.fill_diagonal_(0)
        covariance_matrix2.fill_diagonal_(0)
        
        cov_diff = torch.abs(covariance_matrix1.pow(2) - covariance_matrix2.pow(2))
        return cov_diff.sum() / feature_dim

    covariance_loss_value = covariance_loss(embeddings1, embeddings2)

    total_loss = mu * invariance_loss + lambda_var * variance_loss_value + lambda_cov * covariance_loss_value
    
    return total_loss

In [16]:
class Encoder(torch.nn.Module):
    def __init__(self, model_1, model_2, mlp1, mlp2, aug1, aug2):
        super(Encoder, self).__init__()
        self.model_1 = model_1
        self.model_2 = model_2
        self.mlp1 = mlp1
        self.mlp2 = mlp2
        self.aug1 = aug1
        self.aug2 = aug2
        
    def get_embedding(self, data):
        z, g = self.model_1(data)
        # z_pos, g_pos = self.model_2(data)
        
        z = self.mlp1(z)
        g = self.mlp2(g)
        
        # z_pos = self.mlp1(z_pos)
        # g_pos = self.mlp2(g_pos)

        # g = torch.cat((g, g_pos), 1)
        # z = torch.cat((z, z_pos), 1)
        
        # g = g_pos
        # z = z_pos

        return g.detach(), z.detach()

    def forward(self, data):
        data1 = self.aug1(data.x, data.edge_index, data.y, data.pos, data.edge_attr,
                          data.edge_attr_v2, data.batch, data.ptr)
        data2 = self.aug2(data.x, data.edge_index, data.y, data.pos, data.edge_attr,
                          data.edge_attr_v2, data.batch, data.ptr)
        
        # Structural features
        z1, g1 = self.model_1(data1)
        z2, g2 = self.model_1(data2)
        
        # Positional features
        # z1_pos, g1_pos = self.model_2(data1)
        # z2_pos, g2_pos = self.model_2(data2)
        
        h1, h2 = [self.mlp1(h) for h in [z1, z2]]
        g1, g2 = [self.mlp2(g) for g in [g1, g2]]
        
        # h1_pos, h2_pos = [self.mlp1(h_pos) for h_pos in [z1_pos, z2_pos]]
        # g1_pos, g2_pos = [self.mlp2(g_pos) for g_pos in [g1_pos, g2_pos]]
        
        # h1 = torch.cat((h1, h1_pos), 1)
        # h2 = torch.cat((h2, h2_pos), 1)
        # g1 = torch.cat((g1, g1_pos), 1)
        # g2 = torch.cat((g2, g2_pos), 1)
        
        # h1 = h1_pos
        # h2 = h2_pos
        # g1 = g1_pos
        # g2 = g2_pos
        
        return h1, h2, g1, g2

In [17]:
def train(encoder_model, dataloader, optimizer, device):
    best = float("inf")
    cnt_wait = 0
    best_t = 0
    
    loss_func = NTXentLoss(temperature=0.10)
    
    encoder_model.train()
    epoch_loss = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()

        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)

        h1, h2, g1, g2 = encoder_model(data)
        
        embeddings = torch.cat((g1, g2))
        
        # The same index corresponds to a positive pair
        indices = torch.arange(0, g1.size(0), device=device)
        labels = torch.cat((indices, indices))
        
        reg_loss = vicreg_loss(h1, h2, lambda_var=25, lambda_cov=25, mu=1)
        loss = loss_func(embeddings, labels) + 0.0009*reg_loss
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        
        if epoch % 20 == 0:
            print("Epoch: {0}, Loss: {1:0.4f}".format(epoch, epoch_loss))

        if epoch_loss < best:
            best = epoch_loss
            best_t = epoch
            cnt_wait = 0
            torch.save(encoder_model.state_dict(), './pkl/best_model_'+ args.dataset_name + tag + '.pkl')
        else:
            cnt_wait += 1

        if cnt_wait == args.patience:
            print("Early stopping")
            break
            
    return epoch_loss

In [18]:
def test(encoder_model, dataset, dataloader, device):
    encoder_model.eval()
    data_list = []
    for data in dataset:
        data = data.to(device)
        if data.x is None:
            num_nodes = data.batch.size(0)
            data.x = torch.ones((num_nodes, 1), dtype=torch.float32, device=data.batch.device)
        graph_embeds, node_embeds = encoder_model.get_embedding(data)
        data = Data(x=node_embeds, edge_index=data.edge_index, y=data.y, pos=data.pos, edge_attr=data.edge_attr,
                    edge_attr_v2=data.edge_attr_v2, batch=data.batch, ptr=None)
        data.graph_embeds = graph_embeds
        data_list.append(data)

    val_accs = np.zeros(args.split)
    test_accs = np.zeros(args.split)
    
    for fold, (train_idx, test_idx, val_idx) in enumerate(zip(*train_utils.k_fold(dataset, args.split, args.seed))):

        model = LogReg(hid_dim=data_list[0].graph_embeds.shape[1], out_dim=args.n_classes)
        log.info(f"---------------Training on fold {fold + 1}------------------------")
        model.to(device)
        pytorch_total_params = train_utils.count_parameters(model)
        log.info(f'The total parameters of model :{[pytorch_total_params]}')
        
        train_data_list = []
        valid_data_list = []
        test_data_list = []

        for i in train_idx:
            index = torch.IntTensor.item(i)
            train_data_list.append(data_list[index])

        for i in val_idx:
            index = torch.IntTensor.item(i)
            valid_data_list.append(data_list[index])

        for i in test_idx:
            index = torch.IntTensor.item(i)
            test_data_list.append(data_list[index])

        optimizer = Adam(model.parameters(), lr=args.lr)

        val_loader = loader(valid_data_list, batch_size=args.batch_size, num_workers=args.num_workers)
        test_loader = loader(test_data_list, batch_size=args.batch_size, num_workers=args.num_workers)
        train_loader = loader(train_data_list, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)
        best_val_loss, best_val_acc, best_test_acc, best_train_acc = 100, 0, 0, 0
        start_outer = time.time()
        for epoch in range(args.num_epochs):
            start = time.time()
            train_loss = csl_train(train_loader, model, optimizer, device=device, parallel=args.parallel)
            lr = optimizer.param_groups[0]['lr']
            val_acc = csl_val(val_loader, model, device=device, parallel=args.parallel)
            if val_acc >= best_val_acc:
                test_acc = csl_val(test_loader, model, device=device, parallel=args.parallel)
                best_val_acc = val_acc
                best_test_acc = test_acc

            time_per_epoch = time.time() - start

            log.info('Epoch: {:03d}, LR: {:7f}, Train Loss: {:.7f}, '
                     'Val Acc: {:.7f}, Test Acc: {:.7f}, Best Val Acc: {:.7f}, Best Test Acc: {:.7f}, Seconds: {:.4f}'.format(
                epoch + 1, lr, train_loss, val_acc, test_acc, best_val_acc, best_test_acc, time_per_epoch))

            torch.cuda.empty_cache()  # empty test part memory cost

        time_average_epoch = time.time() - start_outer
        log.info(
            f'Fold {fold + 1}, Vali: {best_val_acc}, Test: {best_test_acc}, Seconds/epoch: {time_average_epoch / (epoch+1)}')
        val_accs[fold] = best_val_acc
        test_accs[fold] = best_test_acc
        
    
    val_accs = torch.tensor(val_accs)
    test_accs = torch.tensor(test_accs)
    
    return val_accs, test_accs

In [19]:
# path = "./data/" + args.dataset_name
path = "./data/GNNBenchmarking"

In [20]:
if os.path.exists(path + '/processed'):
    shutil.rmtree(path + '/processed')

In [21]:
# dataset = SRDataset(path, pre_transform=T.Compose([edge_feature_transform]))
dataset = GNNBenchmarkDataset(root=path, name=args.dataset_name,
                              pre_transform=T.Compose([edge_feature_transform]))

In [22]:
# Determine the maximum degree in the dataset
max_degree = max([data.num_nodes for data in dataset])

# Apply the OneHotDegree transform
dataset.transform = T.OneHotDegree(max_degree)

In [23]:
# Determine the maximum degree in the dataset
max_degree = max([data.num_nodes for data in dataset])

# Apply the OneHotDegree transform
dataset.transform = T.OneHotDegree(max_degree)

In [24]:
args.n_classes = dataset.num_classes

In [25]:
args.input_size = dataset.num_features
args.pos_size = args.pos_enc_dim
args.output_size = args.hidden_size
args.edge_attr_size =  dataset.edge_attr.shape[1]
args.edge_attr_v2_size =  dataset.edge_attr_v2.shape[1]

In [26]:
aug1 = A.Identity()

if args.feature_augmentation:
    # Feature Augmentation
    aug2 = A.RandomChoice([A.FeatureDropout(pf=0.1),
                           A.FeatureMasking(pf=0.1),
                           A.EdgeAttrMasking(pf=0.1)], 1)
else:
    # Structure Augmentation
    aug2 = A.RandomChoice([A.RWSampling(num_seeds=1000, walk_length=10),
                       A.NodeDropping(pn=0.1),
                       A.EdgeRemoving(pe=0.1)], 1)

model_1 = get_model(args)
model_1.to(device)

args.pos_attr = True
args.input_size = args.pos_size
model_2 = get_model(args)
model_2.to(device)

mlp1 = Projection(input_dim=args.hidden_size, output_dim=args.hidden_size)
mlp2 = Projection(input_dim=args.hidden_size, output_dim=args.hidden_size)

encoder_model = Encoder(model_1=model_1, model_2=model_2, mlp1=mlp1, mlp2=mlp2, aug1=aug1, aug2=aug2).to(device)

In [27]:
optimizer = Adam(encoder_model.parameters(), lr=args.lr, weight_decay=args.l2_wd)
dataloader = DataLoader(dataset, batch_size=args.batch_size)
with tqdm(total=10, desc='(T)') as pbar: #10 -> 99.33
    for epoch in range(1, 11): #11 -> 99.33
        loss = train(encoder_model, dataloader, optimizer, device)
        pbar.set_postfix({'loss': loss})
        pbar.update()

(T): 100%|███████████████████████████████████████████| 10/10 [00:11<00:00,  1.13s/it, loss=18]


In [28]:
val_accs, test_accs = test(encoder_model, dataset, dataloader, device)

log.info("-------------------Print final result-------------------------")
log.info(f"Validation result: Mean: {val_accs.mean().item()*100}, Std :{val_accs.std().item()*100}")
log.info(f"Test result: Mean: {test_accs.mean().item()*100}, Std :{test_accs.std().item()*100}")

[10.11.24 16:09:09] ---------------Training on fold 1------------------------
[10.11.24 16:09:09] The total parameters of model :[2010]
[10.11.24 16:09:09] Epoch: 001, LR: 0.001000, Train Loss: 54.6839429, Val Acc: 0.2000000, Test Acc: 0.2000000, Best Val Acc: 0.2000000, Best Test Acc: 0.2000000, Seconds: 0.0045
[10.11.24 16:09:09] Epoch: 002, LR: 0.001000, Train Loss: 27.0371273, Val Acc: 0.3333333, Test Acc: 0.2666667, Best Val Acc: 0.3333333, Best Test Acc: 0.2666667, Seconds: 0.0037
[10.11.24 16:09:09] Epoch: 003, LR: 0.001000, Train Loss: 22.2458986, Val Acc: 0.2000000, Test Acc: 0.2666667, Best Val Acc: 0.3333333, Best Test Acc: 0.2666667, Seconds: 0.0031
[10.11.24 16:09:09] Epoch: 004, LR: 0.001000, Train Loss: 11.8777086, Val Acc: 0.1333333, Test Acc: 0.2666667, Best Val Acc: 0.3333333, Best Test Acc: 0.2666667, Seconds: 0.0032
[10.11.24 16:09:09] Epoch: 005, LR: 0.001000, Train Loss: 11.8125909, Val Acc: 0.2666667, Test Acc: 0.2666667, Best Val Acc: 0.3333333, Best Test Acc: 0

[10.11.24 16:09:09] Epoch: 071, LR: 0.001000, Train Loss: 1.4714136, Val Acc: 0.6666667, Test Acc: 0.6666667, Best Val Acc: 0.7333333, Best Test Acc: 0.6666667, Seconds: 0.0028
[10.11.24 16:09:09] Epoch: 072, LR: 0.001000, Train Loss: 1.7350453, Val Acc: 0.3333333, Test Acc: 0.6666667, Best Val Acc: 0.7333333, Best Test Acc: 0.6666667, Seconds: 0.0028
[10.11.24 16:09:09] Epoch: 073, LR: 0.001000, Train Loss: 1.6729992, Val Acc: 0.6000000, Test Acc: 0.6666667, Best Val Acc: 0.7333333, Best Test Acc: 0.6666667, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 074, LR: 0.001000, Train Loss: 1.4589458, Val Acc: 0.4666667, Test Acc: 0.6666667, Best Val Acc: 0.7333333, Best Test Acc: 0.6666667, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 075, LR: 0.001000, Train Loss: 1.3779167, Val Acc: 0.5333334, Test Acc: 0.6666667, Best Val Acc: 0.7333333, Best Test Acc: 0.6666667, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 076, LR: 0.001000, Train Loss: 1.5669025, Val Acc: 0.6000000, Test Acc: 0.6666667, Best 

[10.11.24 16:09:09] Epoch: 148, LR: 0.001000, Train Loss: 1.2023577, Val Acc: 0.6000000, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0030
[10.11.24 16:09:09] Epoch: 149, LR: 0.001000, Train Loss: 1.1841445, Val Acc: 0.6000000, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0027
[10.11.24 16:09:09] Epoch: 150, LR: 0.001000, Train Loss: 1.3439961, Val Acc: 0.4666667, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 151, LR: 0.001000, Train Loss: 1.2511975, Val Acc: 0.6000000, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0025
[10.11.24 16:09:09] Epoch: 152, LR: 0.001000, Train Loss: 1.1627656, Val Acc: 0.6000000, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 153, LR: 0.001000, Train Loss: 1.1007683, Val Acc: 0.6000000, Test Acc: 0.8000000, Best 

[10.11.24 16:09:09] Epoch: 220, LR: 0.001000, Train Loss: 0.9067299, Val Acc: 0.7333333, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0029
[10.11.24 16:09:09] Epoch: 221, LR: 0.001000, Train Loss: 0.8551071, Val Acc: 0.6000000, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0027
[10.11.24 16:09:09] Epoch: 222, LR: 0.001000, Train Loss: 0.8172937, Val Acc: 0.6666667, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 223, LR: 0.001000, Train Loss: 0.8850718, Val Acc: 0.5333334, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0025
[10.11.24 16:09:09] Epoch: 224, LR: 0.001000, Train Loss: 1.0337372, Val Acc: 0.6666667, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 225, LR: 0.001000, Train Loss: 1.0182050, Val Acc: 0.7333333, Test Acc: 0.7333333, Best 

[10.11.24 16:09:09] Epoch: 295, LR: 0.001000, Train Loss: 0.8127109, Val Acc: 0.8000000, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0029
[10.11.24 16:09:09] Epoch: 296, LR: 0.001000, Train Loss: 1.1322715, Val Acc: 0.6666667, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0027
[10.11.24 16:09:09] Epoch: 297, LR: 0.001000, Train Loss: 0.8792005, Val Acc: 0.6666667, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 298, LR: 0.001000, Train Loss: 1.0160387, Val Acc: 0.7333333, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 299, LR: 0.001000, Train Loss: 0.9492060, Val Acc: 0.7333333, Test Acc: 0.7333333, Best Val Acc: 0.8666667, Best Test Acc: 0.7333333, Seconds: 0.0026
[10.11.24 16:09:09] Epoch: 300, LR: 0.001000, Train Loss: 0.7240392, Val Acc: 0.8666667, Test Acc: 0.7333333, Best 

...

[10.11.24 16:09:22] Epoch: 198, LR: 0.001000, Train Loss: 1.0144881, Val Acc: 0.5333334, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0030
[10.11.24 16:09:22] Epoch: 199, LR: 0.001000, Train Loss: 0.9633706, Val Acc: 0.5333334, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0027
[10.11.24 16:09:22] Epoch: 200, LR: 0.001000, Train Loss: 1.0415776, Val Acc: 0.5333334, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0026
[10.11.24 16:09:22] Epoch: 201, LR: 0.001000, Train Loss: 1.0238511, Val Acc: 0.6666667, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0026
[10.11.24 16:09:22] Epoch: 202, LR: 0.001000, Train Loss: 1.2315167, Val Acc: 0.4666667, Test Acc: 0.8000000, Best Val Acc: 0.8000000, Best Test Acc: 0.8000000, Seconds: 0.0027
[10.11.24 16:09:22] Epoch: 203, LR: 0.001000, Train Loss: 1.2951309, Val Acc: 0.4000000, Test Acc: 0.8000000, Best 

[10.11.24 16:09:22] Epoch: 273, LR: 0.001000, Train Loss: 0.8924847, Val Acc: 0.6666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0028
[10.11.24 16:09:22] Epoch: 274, LR: 0.001000, Train Loss: 0.9985387, Val Acc: 0.4666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0027
[10.11.24 16:09:22] Epoch: 275, LR: 0.001000, Train Loss: 0.9404663, Val Acc: 0.6000000, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0026
[10.11.24 16:09:22] Epoch: 276, LR: 0.001000, Train Loss: 1.1681466, Val Acc: 0.4666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0026
[10.11.24 16:09:22] Epoch: 277, LR: 0.001000, Train Loss: 1.1111309, Val Acc: 0.6666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0027
[10.11.24 16:09:22] Epoch: 278, LR: 0.001000, Train Loss: 0.9378940, Val Acc: 0.5333334, Test Acc: 0.8666667, Best 

[10.11.24 16:09:22] Epoch: 349, LR: 0.001000, Train Loss: 0.7540376, Val Acc: 0.8000000, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0029
[10.11.24 16:09:22] Epoch: 350, LR: 0.001000, Train Loss: 0.7624434, Val Acc: 0.8000000, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0028
[10.11.24 16:09:22] Epoch: 351, LR: 0.001000, Train Loss: 0.8537097, Val Acc: 0.7333333, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0026
[10.11.24 16:09:22] Epoch: 352, LR: 0.001000, Train Loss: 0.7827993, Val Acc: 0.6666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0027
[10.11.24 16:09:22] Epoch: 353, LR: 0.001000, Train Loss: 0.6784940, Val Acc: 0.6666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0027
[10.11.24 16:09:22] Epoch: 354, LR: 0.001000, Train Loss: 0.6848293, Val Acc: 0.6666667, Test Acc: 0.8666667, Best 

[10.11.24 16:09:22] Epoch: 424, LR: 0.001000, Train Loss: 0.9292881, Val Acc: 0.8000000, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0031
[10.11.24 16:09:22] Epoch: 425, LR: 0.001000, Train Loss: 0.5878759, Val Acc: 0.8666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0035
[10.11.24 16:09:22] Epoch: 426, LR: 0.001000, Train Loss: 0.8764130, Val Acc: 0.6666667, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0031
[10.11.24 16:09:22] Epoch: 427, LR: 0.001000, Train Loss: 1.1497090, Val Acc: 0.7333333, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0029
[10.11.24 16:09:22] Epoch: 428, LR: 0.001000, Train Loss: 1.1102583, Val Acc: 0.6000000, Test Acc: 0.8666667, Best Val Acc: 0.8666667, Best Test Acc: 0.8666667, Seconds: 0.0029
[10.11.24 16:09:22] Epoch: 429, LR: 0.001000, Train Loss: 0.8261546, Val Acc: 0.8000000, Test Acc: 0.8666667, Best 

[10.11.24 16:09:23] Epoch: 494, LR: 0.001000, Train Loss: 0.7465530, Val Acc: 0.5333334, Test Acc: 0.9333333, Best Val Acc: 0.9333333, Best Test Acc: 0.9333333, Seconds: 0.0029
[10.11.24 16:09:23] Epoch: 495, LR: 0.001000, Train Loss: 0.7543444, Val Acc: 0.4666667, Test Acc: 0.9333333, Best Val Acc: 0.9333333, Best Test Acc: 0.9333333, Seconds: 0.0028
[10.11.24 16:09:23] Epoch: 496, LR: 0.001000, Train Loss: 0.8086891, Val Acc: 0.6000000, Test Acc: 0.9333333, Best Val Acc: 0.9333333, Best Test Acc: 0.9333333, Seconds: 0.0028
[10.11.24 16:09:23] Epoch: 497, LR: 0.001000, Train Loss: 0.9768159, Val Acc: 0.8666667, Test Acc: 0.9333333, Best Val Acc: 0.9333333, Best Test Acc: 0.9333333, Seconds: 0.0027
[10.11.24 16:09:23] Epoch: 498, LR: 0.001000, Train Loss: 1.0167892, Val Acc: 0.6000000, Test Acc: 0.9333333, Best Val Acc: 0.9333333, Best Test Acc: 0.9333333, Seconds: 0.0026
[10.11.24 16:09:23] Epoch: 499, LR: 0.001000, Train Loss: 1.2069150, Val Acc: 0.7333333, Test Acc: 0.9333333, Best 